In [186]:
import os
import pandas as pd
import geopandas as gpd
from collections import defaultdict
import csv

full_county_map_file = "../data/mappings/Complete_Counties_in_2022_Assembly.csv"
results_dir = "../data/2020_Primary_03"
statewide_results_file = os.path.join(results_dir, "Statewide", "statewide_sov_by_county.csv")

county_mappings = {r['County']: r['District'] for r in csv.DictReader(open(full_county_map_file))}
statewide_results = pd.read_csv(statewide_results_file, encoding='ISO-8859-1')

big3 = ['JOSEPH R. BIDEN', 'ELIZABETH WARREN', 'BERNIE SANDERS']

In [187]:
results = statewide_results[statewide_results.CONTEST_NAME == 'President Democratic'][statewide_results.COUNTY_NAME != 'State Totals'] #TODO
total_results = defaultdict(dict)

for n,v in results.groupby(["COUNTY_NAME", 'CANDIDATE_NAME']):
    total_results[n[0]][n[1]] = int(v['VOTE_TOTAL'])

/var/folders/9w/7ntvb0w57mnbkqngqfmkmzc40000gq/T/ipykernel_95014/618470236.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  results = statewide_results[statewide_results.CONTEST_NAME == 'President Democratic'][statewide_results.COUNTY_NAME != 'State Totals'] #TODO


In [188]:
district_results = {dist_num+1: defaultdict(int) for dist_num in range(80)}
for county_name, district in county_mappings.items():
    print(county_name, district)
    county_results = total_results[county_name]
    for candidate, votes in county_results.items():
        district_results[int(district)][candidate.upper()] += votes
        district_results[int(district)]["BALLOTS CAST"] += votes

Alpine 1
Butte 3
Colusa 4
Del Norte 2
Glenn 3
Humboldt 2
Imperial 36
Inyo 8
Kings 33
Lake 4
Lassen 1
Marin 12
Mariposa 8
Mendocino 2
Modoc 1
Mono 8
Napa 4
Nevada 1
Plumas 1
San Benito 29
Santa Barbara 37
Shasta 1
Sierra 1
Siskiyou 1
Solano 11
Sutter 3
Tehama 3
Trinity 2
Tuolumne 8
Yolo 4
Yuba 3


In [189]:
dirname = "../data/2020_Primary_03/reshaped"
for fname in os.listdir(dirname):
    print(fname)
    if "swp" not in fname:
        with open(os.path.join(dirname, fname)) as f:
            for row in csv.DictReader(f):
                dist = int(row['district'])
                for key in row:
                    if key != "district":
                        district_results[dist][key] += float(row[key])

Merced_County.csv
Tulare_county.csv
SLO_County.csv
Monterey_County.csv
San_Francisco_County.csv
Fresno_County.csv
Sonoma_County.csv
San_Joaquin_County.csv
Alameda_County.csv
Sacramento_County.csv
San_Diego_County.csv
San_Mateo_County.csv
Orange_County.csv
Los_Angeles_County.csv
Stanislaus_County.csv
Riverside_County.csv
Contra_Costa_County.csv
SB_County.csv
Santa_Clara_County.csv
Ventura_County.csv
Kern_County.csv
Santa_Cruz_County.csv


In [191]:
outfile = f"../data/2020_Primary_03/Statewide/Statewide_By_2022_Assembly.csv"
results_cols = ['JOSEPH R. BIDEN', 'ELIZABETH WARREN', 'BERNIE SANDERS']
write_cols = ["district", 'BALLOTS CAST'] + results_cols
writer = csv.DictWriter(open(outfile, "w", newline=""), write_cols)
writer.writeheader()
for district, results in district_results.items():
    results['district'] = district
    data = {k: results.get(k,0) for k in write_cols}
    writer.writerow(data)

In [ ]:
# Orange County
county = "Orange"
oc_results_file = f"../data/2020_Primary_03/{county}_county/{county}_County.csv"
oc_res = pd.read_csv(oc_results_file, skiprows=1, encoding='ISO-8859-1', dtype={'Precinct': str})

oc_mapping_file = f"../data/mappings/{county}_County_March_2020_as_2022_Assembly.csv"
oc_mappings = {r['precinct']: r['District'] for r in csv.DictReader(open(oc_mapping_file))}

county_results = defaultdict(lambda: defaultdict(int))
for (precinct, candidate), votes in  oc_res[oc_res['Contest Party'] == 'DEM'][oc_res['Contest Title'] == 'PRESIDENT OF THE UNITED STATES'].groupby(['Precinct', 'Choice Name']):
    district = int(oc_mappings[precinct])
    if 'write-ins' not in candidate:
        count = int(votes['Total Votes'])
        if count > 0:
            county_results[district][candidate] += count
            county_results[district]['BALLOTS CAST'] += count
            
outfile = f"../data/2020_Primary_03/reshaped/{county}_County.csv"
results_cols = ['JOSEPH R. BIDEN', 'ELIZABETH WARREN', 'BERNIE SANDERS']
write_cols = ["district", 'BALLOTS CAST'] + results_cols
writer = csv.DictWriter(
    open(outfile, "w", newline=""), write_cols
)
writer.writeheader()
for district, results in county_results.items():
    results['district'] = district
    data = {k: results[k] for k in write_cols}
    print(district, data)
    writer.writerow(data)

/var/folders/9w/7ntvb0w57mnbkqngqfmkmzc40000gq/T/ipykernel_95014/961940633.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  for (precinct, candidate), votes in  oc_res[oc_res['Contest Party'] == 'DEM'][oc_res['Contest Title'] == 'PRESIDENT OF THE UNITED STATES'].groupby(['Precinct', 'Choice Name']):


68 {'district': 68, 'BALLOTS CAST': 46953, 'JOSEPH R. BIDEN': 9169, 'ELIZABETH WARREN': 3786, 'BERNIE SANDERS': 25689}
59 {'district': 59, 'BALLOTS CAST': 46719, 'JOSEPH R. BIDEN': 14955, 'ELIZABETH WARREN': 5305, 'BERNIE SANDERS': 14489}
67 {'district': 67, 'BALLOTS CAST': 41585, 'JOSEPH R. BIDEN': 10388, 'ELIZABETH WARREN': 3597, 'BERNIE SANDERS': 18249}
70 {'district': 70, 'BALLOTS CAST': 44189, 'JOSEPH R. BIDEN': 11206, 'ELIZABETH WARREN': 3713, 'BERNIE SANDERS': 18065}
64 {'district': 64, 'BALLOTS CAST': 6851, 'JOSEPH R. BIDEN': 2059, 'ELIZABETH WARREN': 565, 'BERNIE SANDERS': 2774}
72 {'district': 72, 'BALLOTS CAST': 78302, 'JOSEPH R. BIDEN': 24769, 'ELIZABETH WARREN': 9021, 'BERNIE SANDERS': 21137}
74 {'district': 74, 'BALLOTS CAST': 30316, 'JOSEPH R. BIDEN': 9984, 'ELIZABETH WARREN': 3334, 'BERNIE SANDERS': 7924}
71 {'district': 71, 'BALLOTS CAST': 27239, 'JOSEPH R. BIDEN': 8878, 'ELIZABETH WARREN': 3062, 'BERNIE SANDERS': 7932}
73 {'district': 73, 'BALLOTS CAST': 61553, 'JOSEP

In [136]:
county = "Ventura"
county_results_file = f"../data/2020_Primary_03/{county}_County/{county}_County_sov_1.csv"
#county_res = pd.read_csv(county_results_file, encoding='ISO-8859-1', dtype={'Precinct': str})
county_res = list(csv.reader(open(county_results_file, encoding='utf-8-sig')))

county_mapping_file = f"../data/mappings/{county}_County_March_2020_as_2022_Assembly.csv"
county_mappings = {r['precinct']: r['District'] for r in csv.DictReader(open(county_mapping_file))}
county_res[1]

['Precinct',
 'JULIÁN CASTRO (DEM)',
 '',
 'MICHAEL A. ELLINGER (DEM)',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

In [133]:
county_results = defaultdict(lambda: defaultdict(int))
last_prec = None
for row in county_res:
    if 'Precinct' in row[0]:
        row = [r.strip() for r in row if r != '']
        keys = row
    elif str(row[0]) in county_mappings:
        row = [r.strip() for r in row if r != '']
        dist = county_mappings[row[0]]
        for k, v in zip(keys, row):
            try:
                if k == 'Party Member Ballots Cast':
                    k = 'Ballots Cast'
                county_results[dist][k.upper()] += int(v.replace(",", "").strip())
            except ValueError as e:
                if k == 'JOSEPH R. BIDEN':
                    print(list(zip(keys, row)))
    else:
        #$print(row[0])
        pass
county_results

defaultdict(<function __main__.<lambda>()>,
            {'42': defaultdict(int,
                         {'PRECINCT': 74579000, 'TIMES CAST': 14527750}),
             '46': defaultdict(int, {'PRECINCT': 260000, 'TIMES CAST': 26000}),
             '38': defaultdict(int,
                         {'PRECINCT': 99121590, 'TIMES CAST': 19824318})})

In [40]:
outfile = f"../data/2020_Primary_03/reshaped/{county}_County.csv"
results_cols = ['JOSEPH R. BIDEN', 'ELIZABETH WARREN', 'BERNIE SANDERS']
write_cols = ["district", 'BALLOTS CAST'] + results_cols
writer = csv.DictWriter(open(outfile, "w", newline=""), write_cols)
writer.writeheader()
for district, results in county_results.items():
    results['district'] = district
    data = {k: results[k] for k in write_cols}
    writer.writerow(data)

In [ ]:
for prec, results in cleaned_results.items():
    if prec is None:
        continue
    precinct = prec.split(' ')[1]
    if precinct not in county_mappings:
        precinct = precinct.split('/')[0]
    if precinct in county_mappings:
        district = county_mappings[precinct]
        for key, val in results.items():
            if '(DEM)' in key:
                candidate = key.replace(" (DEM) ", "")
                county_results[district][candidate] += int(val)
            elif key == 'Total Votes':
                county_results[district]["BALLOTS CAST"] += int(val)

In [92]:
outfile = f"../data/2020_Primary_03/reshaped/{county}_County.csv"
results_cols = ['JOSEPH R. BIDEN', 'ELIZABETH WARREN', 'BERNIE SANDERS']
write_cols = ["district", 'BALLOTS CAST'] + results_cols
writer = csv.DictWriter(open(outfile, "w", newline=""), write_cols)
writer.writeheader()
for district, results in county_results.items():
    results['district'] = district
    data = {k: results[k] for k in write_cols}
    writer.writerow(data)

In [173]:
county = "Riverside"
county_results_file = f"../data/2020_Primary_03/{county}_County/{county}_County_sov.csv"
county_res = pd.read_csv(county_results_file, dtype={'Precinct': str})
#county_res = list(csv.DictReader(open(county_results_file, encoding='ISO-8859-1')))

county_mapping_file = f"../data/mappings/{county}_County_March_2020_as_2022_Assembly.csv"
county_mappings = {int(r['precinct']): r['District'] for r in csv.DictReader(open(county_mapping_file))}

In [180]:
county_results = defaultdict(lambda: defaultdict(int))
for district, row in county_res.iterrows():
    if row['Precinct'] == 'Countywide - Total':
        continue
    if int(row.Precinct) in county_mappings:
        district = county_mappings[int(row.Precinct)]
        for k in county_res.columns[1:]:
            try:
                name = k.replace("(DEM)", "").strip()
                if name == 'Total Votes':
                    name = "BALLOTS CAST"
                v = int(str(row[k]).strip())
                county_results[district][name] += v
            except ValueError as e:
                pass
    else:
        print(row.Precinct)
county_results

defaultdict(<function __main__.<lambda>()>,
            {'60': defaultdict(int,
                         {'Unnamed: 1': 37554,
                          'Countywide': 5846408,
                          'JULIÁN CASTRO': 228,
                          'MICHAEL A. ELLINGER': 47,
                          'ELIZABETH WARREN': 2068,
                          'MARIANNE WILLIAMSON': 31,
                          'ANDREW YANG': 203,
                          'ROQUE "ROCKY" DE LA FUENTE III': 54,
                          'JOHN K. DELANEY': 62,
                          'AMY KLOBUCHAR': 387,
                          'BERNIE SANDERS': 16424,
                          'JOE SESTAK': 35,
                          'TOM STEYER': 1085,
                          'DEVAL PATRICK': 17,
                          'MICHAEL BENNET': 54,
                          'MICHAEL R. BLOOMBERG': 4326,
                          'MOSIE BOYD': 16,
                          'CORY BOOKER': 52,
                          'PET

In [182]:
outfile = f"../data/2020_Primary_03/reshaped/{county}_County.csv"
results_cols = ['JOSEPH R. BIDEN', 'ELIZABETH WARREN', 'BERNIE SANDERS']
write_cols = ["district", 'BALLOTS CAST'] + results_cols
writer = csv.DictWriter(open(outfile, "w", newline=""), write_cols)
writer.writeheader()
for district, results in county_results.items():
    results['district'] = district
    data = {k: results[k] for k in write_cols}
    print(data)
    writer.writerow(data)

{'district': '60', 'BALLOTS CAST': 36634, 'JOSEPH R. BIDEN': 10433, 'ELIZABETH WARREN': 2068, 'BERNIE SANDERS': 16424}
{'district': '58', 'BALLOTS CAST': 40268, 'JOSEPH R. BIDEN': 10903, 'ELIZABETH WARREN': 3170, 'BERNIE SANDERS': 18822}
{'district': '63', 'BALLOTS CAST': 49699, 'JOSEPH R. BIDEN': 15961, 'ELIZABETH WARREN': 4140, 'BERNIE SANDERS': 18293}
{'district': '71', 'BALLOTS CAST': 26424, 'JOSEPH R. BIDEN': 8449, 'ELIZABETH WARREN': 2546, 'BERNIE SANDERS': 8953}
{'district': '47', 'BALLOTS CAST': 44930, 'JOSEPH R. BIDEN': 17129, 'ELIZABETH WARREN': 5135, 'BERNIE SANDERS': 12779}
{'district': '36', 'BALLOTS CAST': 20807, 'JOSEPH R. BIDEN': 6645, 'ELIZABETH WARREN': 1493, 'BERNIE SANDERS': 8282}


In [65]:
county_results = defaultdict(lambda: defaultdict(int))

prec = None
for _,row in county_res.iterrows():
    try:
        prec = int(row.Precinct)
    except ValueError:
        if row.Precinct == 'Total':
            for k in county_res.columns:
                try:
                    county_results[county_mappings[str(prec)]][k.strip()] += int(row[k])
                    county_results[county_mappings[str(prec)]]['BALLOTS CAST'] += int(row[k])
                except ValueError:
                    pass
        if row.Precinct == 'Cumulative':
            break
county_results

defaultdict(<function __main__.<lambda>()>, {})

In [383]:
outfile = f"../data/2020_Primary_03/reshaped/{county}_County.csv"
results_cols = ['JOSEPH R. BIDEN', 'ELIZABETH WARREN', 'BERNIE SANDERS']
write_cols = ["district", 'BALLOTS CAST'] + results_cols
writer = csv.DictWriter(
    open(outfile, "w", newline=""), write_cols
)
writer.writeheader()
for district, results in county_results.items():
    results['district'] = district
    data = {k: results[k] for k in write_cols}
    writer.writerow(data)

In [78]:
county = "Merced"
county_results_file = f"../data/2020_Primary_03/{county}_County/{county}_County_sov.csv"
#county_res = pd.read_csv(county_results_file, dtype={'PRECINCT_NAME': str})
county_res = list(csv.DictReader(open(county_results_file, encoding='utf-8-sig')))

county_mapping_file = f"../data/mappings/{county}_County_March_2020_as_2022_Assembly.csv"
county_mappings = {r['precinct']: r['District'] for r in csv.DictReader(open(county_mapping_file))}
county_res[0]

FileNotFoundError: [Errno 2] No such file or directory: '../data/mappings/Merced_County_March_2020_as_2022_Assembly.csv'

In [77]:
county_results = defaultdict(lambda: defaultdict(int))
prec = None
for row in county_res:
    if row['Precinct'] in county_mappings:
        prec = row['Precinct']
    elif row['Precinct'] == 'Total':
        district = county_mappings[prec]
        for k, v in row.items():
            name = None
            if '(DEM)' in k:
                name = k.replace("(DEM)","").strip()
            elif k == 'Total Votes':
                name = 'BALLOTS CAST'
            if name is not None and '*' not in v:
                county_results[district][name] += int(v)
outfile = f"../data/2020_Primary_03/reshaped/{county}_County.csv"
results_cols = ['JOSEPH R. BIDEN', 'ELIZABETH WARREN', 'BERNIE SANDERS']
write_cols = ["district", 'BALLOTS CAST'] + results_cols
writer = csv.DictWriter(
    open(outfile, "w", newline=""), write_cols
)
writer.writeheader()
for district, results in county_results.items():
    results['district'] = district
    data = {k: results[k] for k in write_cols}
    writer.writerow(data)
county_results

defaultdict(<function __main__.<lambda>()>,
            {'29': defaultdict(int,
                         {'JULIÁN CASTRO': 147,
                          'MICHAEL A. ELLINGER': 17,
                          'ELIZABETH WARREN': 1860,
                          'MARIANNE WILLIAMSON': 27,
                          'ANDREW YANG': 166,
                          'ROQUE "ROCKY" DE LA FUENTE III': 21,
                          'JOHN K. DELANEY': 17,
                          'AMY KLOBUCHAR': 419,
                          'BERNIE SANDERS': 12007,
                          'JOE SESTAK': 38,
                          'TOM STEYER': 913,
                          'DEVAL PATRICK': 12,
                          'MICHAEL BENNET': 57,
                          'MICHAEL R. BLOOMBERG': 3052,
                          'MOSIE BOYD': 12,
                          'CORY BOOKER': 22,
                          'PETE BUTTIGIEG': 862,
                          'JOSEPH R. BIDEN': 6673,
                          '

In [53]:
county_results = defaultdict(lambda: defaultdict(int))
grouped = county_res[county_res['CONTEST_FULL_NAME'] == 'President Of The United States-Democratic'].groupby(['PRECINCT_NAME', 'CANDIDATE_FULL_NAME'])
nonstandard_precincts_SLO = {
    'MB01': 'M1',
    'MB02': 'M2',
    'MB03': 'M3',
    'MB04': 'M4',
    'MB05': 'M5',
}
for (precinct, candidate), votes in grouped:
    if precinct not in county_mappings:
        if precinct in nonstandard_precincts_SLO:
            district = int(county_mappings[nonstandard_precincts_SLO[precinct]])
        else:
            # should be ok since MB06 doesn't have any registered voters
            continue
    else:
        district = int(county_mappings[precinct])
        
    if 'write-ins' not in candidate:
        count = int(votes.sum()['VOTE_COUNT'])
        if count > 0:
            county_results[district][candidate] += count
            county_results[district]['BALLOTS CAST'] += count

In [54]:
outfile = f"../data/2020_Primary_03/reshaped/{county}_County.csv"
results_cols = ['JOSEPH R. BIDEN', 'ELIZABETH WARREN', 'BERNIE SANDERS']
write_cols = ["district", 'BALLOTS CAST'] + results_cols
writer = csv.DictWriter(open(outfile, "w", newline=""), write_cols)
writer.writeheader()
for district, results in county_results.items():
    results['district'] = district
    data = {k: results[k] for k in write_cols}
    writer.writerow(data)

71

In [118]:
county = "Fresno"
county_results_file = f"../data/2020_Primary_03/{county}_County/{county}_County_sov.csv"
#county_res = pd.read_csv(county_results_file, encoding='ISO-8859-1', dtype={'Precinct': str})
county_res = list(csv.DictReader(open(county_results_file, encoding='utf-8-sig')))

county_mapping_file = f"../data/mappings/{county}_County_March_2020_as_2022_Assembly.csv"
county_mappings = {r['precinct']: r['District'] for r in csv.DictReader(open(county_mapping_file))}
county_res[0]


{'Precinct': '0000001',
 '': '',
 'Electionwide': '0000001',
 'JULIÁN CASTRO\n(DEM) ': '',
 'MICHAEL A. ELLINGER\n(DEM) ': '',
 'ELIZABETH WARREN\n(DEM) ': '',
 'MARIANNE WILLIAMSON\n(DEM) ': '',
 'ANDREW YANG\n(DEM) ': '',
 'ROQUE "ROCKY " DE LA FUENTE III\n(DEM) ': '',
 'JOHN K. DELANEY\n(DEM) ': '',
 'AMY KLOBUCHAR\n(DEM) ': '',
 'BERNIE SANDERS\n(DEM) ': '',
 'JOE SESTAK\n(DEM) ': '',
 'TOM STEYER\n(DEM) ': '',
 'DEVAL PATRICK\n(DEM) ': '',
 'MICHAEL BENNET\n(DEM) ': '',
 'MICHAEL R. BLOOMBERG\n(DEM) ': '',
 'MOSIE BOYD\n(DEM) ': '',
 'CORY BOOKER\n(DEM) ': '',
 'PETE BUTTIGIEG\n(DEM) ': '',
 'JOSEPH R. BIDEN\n(DEM) ': '',
 'TULSI GABBARD\n(DEM) ': '',
 'MARK STEWART GREENSTEIN\n(DEM) ': '',
 'Write-in\n ': '',
 'Total Votes': ''}

In [127]:
county_results = defaultdict(lambda: defaultdict(int))

prec = None
for row in county_res:
    try:
        prec = int(row['Precinct'])
    except ValueError:
        if row['Precinct'] == 'Total':
            for k,v in row.items():
                try:
                    name = k.replace("(DEM)", "").strip()
                    if name == 'Total Votes':
                        name = 'BALLOTS CAST'
                    county_results[county_mappings[str(prec)]][name] += int(row[k])
                except ValueError:
                    pass
        if row['Precinct'] == 'Cumulative':
            break
county_results

defaultdict(<function __main__.<lambda>()>,
            {'31': defaultdict(int,
                         {'Precinct': 0,
                          '': 0,
                          'Electionwide': 0,
                          'JULIÁN CASTRO': 161,
                          'MICHAEL A. ELLINGER': 22,
                          'ELIZABETH WARREN': 2756,
                          'MARIANNE WILLIAMSON': 36,
                          'ANDREW YANG': 637,
                          'ROQUE "ROCKY " DE LA FUENTE III': 64,
                          'JOHN K. DELANEY': 36,
                          'AMY KLOBUCHAR': 492,
                          'BERNIE SANDERS': 15791,
                          'JOE SESTAK': 36,
                          'TOM STEYER': 2010,
                          'DEVAL PATRICK': 20,
                          'MICHAEL BENNET': 71,
                          'MICHAEL R. BLOOMBERG': 4881,
                          'MOSIE BOYD': 21,
                          'CORY BOOKER': 62,
      

In [129]:
outfile = f"../data/2020_Primary_03/reshaped/{county}_County.csv"
write_cols = ["district", 'BALLOTS CAST'] + big3
writer = csv.DictWriter(open(outfile, "w", newline=""), write_cols)
writer.writeheader()
for district, results in county_results.items():
    results['district'] = district
    data = {k: results[k] for k in write_cols}
    writer.writerow(data)

In [142]:
county = "Ventura"
county_results_file = f"../data/2020_Primary_03/{county}_County/{county}_County_sov_1.csv"
#county_res = pd.read_csv(county_results_file, encoding='ISO-8859-1', dtype={'Precinct': str})
county_res = list(csv.reader(open(county_results_file, encoding='utf-8-sig')))

county_mapping_file = f"../data/mappings/{county}_County_March_2020_as_2022_Assembly.csv"
county_mappings = {r['precinct']: r['District'] for r in csv.DictReader(open(county_mapping_file))}
county_res[1]

['Precinct',
 'JULIÁN CASTRO (DEM)',
 '',
 'MICHAEL A. ELLINGER (DEM)',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

In [143]:
county_results = defaultdict(lambda: defaultdict(int))
prec = None
keys = []
for row in county_res:
    if row[0] == 'Precinct':
        keys = row
    elif row[0] in county_mappings:
        prec = row[0]
    elif row[0] == 'Total':
        district = county_mappings[prec]
        for k, v in zip(keys, row):
            name = None
            if '(DEM)' in k:
                name = k.replace("(DEM)","").strip()
            elif k == 'Total Votes':
                name = 'BALLOTS CAST'
            if name is not None and '*' not in v:
                county_results[district][name] += int(v)

county_results

defaultdict(<function __main__.<lambda>()>,
            {'42': defaultdict(int,
                         {'JULIÁN CASTRO': 24,
                          'MICHAEL A. ELLINGER': 8,
                          'ELIZABETH WARREN': 7468,
                          'MARIANNE WILLIAMSON': 52,
                          'ANDREW YANG': 371,
                          'ROQUE "ROCKY" DE LA FUENTE III': 27,
                          'JOHN K. DELANEY': 30,
                          'AMY KLOBUCHAR': 1584,
                          'BERNIE SANDERS': 17700,
                          'JOE SESTAK': 14,
                          'TOM STEYER': 1027,
                          'DEVAL PATRICK': 10,
                          'MICHAEL BENNET': 39,
                          'MICHAEL R. BLOOMBERG': 7810,
                          'MOSIE BOYD': 10,
                          'CORY BOOKER': 38,
                          'PETE BUTTIGIEG': 2912,
                          'JOSEPH R. BIDEN': 20869,
                         

In [146]:
outfile = f"../data/2020_Primary_03/reshaped/{county}_County.csv"
results_cols = ['JOSEPH R. BIDEN', 'ELIZABETH WARREN', 'BERNIE SANDERS']
write_cols = ["district", 'BALLOTS CAST'] + results_cols
writer = csv.DictWriter(
    open(outfile, "w", newline=""), write_cols
)
writer.writeheader()
for district, results in county_results.items():
    results['district'] = district
    data = {k: results[k] for k in write_cols}
    writer.writerow(data)

In [119]:
county_results = defaultdict(lambda: defaultdict(int))
totals = county_res.copy()
for i, row in totals.iterrows():
    if row['Precinct'] == 'Total':
        totals.iat[i, 0] = totals.iat[i-3, 0]
totals = totals[totals['Precinct.1'] == 'Total']
totals = totals[totals['Precinct'] != 'Cumulative'] # ignore total count row

for i, row in totals.iterrows():
    precinct = row['Precinct'].split(' ')[0]
    district = int(county_mappings[precinct])
    
    for candidate in big3:
        if '*' in row[candidate]:
            count = 0
        else:
            count = int(row[candidate])
        if count > 0:
            county_results[district][candidate] += count
            county_results[district]['BALLOTS CAST'] += count

AttributeError: 'list' object has no attribute 'iterrows'

In [179]:
outfile = f"../data/2020_Primary_03/reshaped/{county}_County.csv"
write_cols = ["district", 'BALLOTS CAST'] + big3
writer = csv.DictWriter(open(outfile, "w", newline=""), write_cols)
writer.writeheader()
for district, results in county_results.items():
    results['district'] = district
    data = {k: results[k] for k in write_cols}
    writer.writerow(data)

In [ ]:
county = "Placer"
county_results_file = f"../data/2020_Primary_03/{county}_County/{county}_County_sov.xlsx" #TODO convert xlsx to csv format or read_excel
#county_res = pd.read_csv(county_results_file, dtype={'PRECINCT_NAME': str})

#county_mapping_file = f"../data/mappings/{county}_County_March_2020_as_2022_Assembly.csv"
#county_mappings = {r['precinct']: r['District'] for r in csv.DictReader(open(county_mapping_file))}
#county_res